## Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
from io import StringIO
import time
import openpyxl
from mortgage import Loan

## Functions

Returns a list containing the requested urls.

In [2]:
def list_urls(start, stop, time_delta): #start/stop: YYMM format, time delta: m for monthly & y for yearly
    
    #create empty list for urls
    url_list = []
    
    #create datetime objs for start/stop
    start_date = datetime.date.fromisoformat('20{}-{}-01'.format(start[:2], start[2:]))
    stop_date = datetime.date.fromisoformat('20{}-{}-01'.format(stop[:2], stop[2:])) 
    
    #create datetime obj equal to start_date for use in loop
    current_date = start_date

    #elif loop for each time_delta & while loop for appending to url_list
    if time_delta == 'm': #shows only the BP data for each month of the specified time span
        while (current_date <= stop_date):
            url_list.append("https://www2.census.gov/econ/bps/County/co{}c.txt".format(current_date.strftime("%y%m")))
            current_date = current_date + relativedelta(months = +1)
    
    elif time_delta == 'y': #Cumulative BP data for each respective year from Jan. up to specified starting month.
        while (current_date <= stop_date):
            url_list.append("https://www2.census.gov/econ/bps/County/co{}y.txt".format(current_date.strftime("%y%m")))
            current_date = current_date + relativedelta(years = +1)
    
    else:
        url_list = "invaild time delta"
    
    #return complete url_list
    return url_list

In [3]:
url_L1 = list_urls('1612', '2112', 'y')

Function that creates a single data frame containing all data from each url in the passed url list

In [4]:
def building_permit_df(url_list):
    bp_df = pd.DataFrame()
   
    keys = ['Survey_Date', 'State_FIPS', 'County_FIPS', 'Region_Code', 'Division_Code', 'County_Name',
       '1_unit_Bldgs', '1_unit_Units', '1_unit_Value',
        '2_units_Bldgs', '2_units_Units', '2_units_Value',
        '3-4_units_Bldgs', '3-4_units_Units', '3-4_units_Value',
        '5+_units_Bldgs', '5+_units_Units', '5+_units_Value',
        '1_unit_rep_Bldgs', '1_unit_rep_Units', '1_unit_rep_Value',
        '2_units_rep_Bldgs', '2_units_rep_Units', '2_units_rep_Value',
        '3-4_units_rep_Bldgs', '3-4_units_rep_Units', '3-4_units_rep_Value',
        '5+units_rep_Bldgs', '5+units_rep_Units', '5+units_rep_Value']
    
    for url in url_list:
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'lxml')
        bp_text = soup.get_text()
        bp_textfile = StringIO(bp_text)
        bp_data = pd.read_csv(bp_textfile, names = keys, sep=r'\,|\t', engine='python', skiprows= 3)
        bp_df = pd.concat([bp_df, bp_data], ignore_index=True)
                          
    return bp_df

In [5]:
bp_df1 = pd.DataFrame()
bp_df1 = building_permit_df(url_L1).copy()

In [6]:
bp_df1.to_csv('bp1.csv')

In [7]:
xlsx = pd.ExcelFile('Income.xlsx')

in16 = pd.read_excel(xlsx, '2016')
in17 = pd.read_excel(xlsx, '2017')
in18 = pd.read_excel(xlsx, '2018')
in19 = pd.read_excel(xlsx, '2019')
in20 = pd.read_excel(xlsx, '2020')
in21 = pd.read_excel(xlsx, '2021')

in16['Mean Monthly Income'] = in16['Mean Income'] / 12
in16['Mean Monthly Income'] = in16['Mean Monthly Income'].astype(int)

in17['Mean Monthly Income'] = in17['Mean Income'] / 12
in17['Mean Monthly Income'] = in17['Mean Monthly Income'].astype(int)

in18['Mean Monthly Income'] = in18['Mean Income'] / 12
in18['Mean Monthly Income'] = in18['Mean Monthly Income'].astype(int)

in19['Mean Monthly Income'] = in19['Mean Income'] / 12
in19['Mean Monthly Income'] = in19['Mean Monthly Income'].astype(int)

in20['Mean Monthly Income'] = in20['Mean Income'] / 12
in20['Mean Monthly Income'] = in20['Mean Monthly Income'].astype(int)

in21['Mean Monthly Income'] = in21['Mean Income'] / 12
in21['Mean Monthly Income'] = in21['Mean Monthly Income'].astype(int)

In [8]:
bp_df2 = bp_df1[['Survey_Date', 'State_FIPS','County_FIPS','County_Name','1_unit_Units', '1_unit_Value']].copy()
bp_df2['Mean_unit_Value'] = bp_df2['1_unit_Value'] / bp_df2['1_unit_Units']
bp_df2 = bp_df2.dropna()
bp_df2['Survey_Date'] = bp_df2['Survey_Date'].astype(str).str[:-2].astype(int)

In [9]:
#mortgage calculation
mortgage_principal = bp_df2['Mean_unit_Value']*0.8

monthly_mortgage = []

for value in mortgage_principal:
    monthly_mortgage.append(int(Loan(principal= value, interest=.0375, term=30).monthly_payment))

bp_df2['Monthly_Mortgage'] = monthly_mortgage

In [10]:
#adding year and state fips numbers to income dataframes
in16['Year'] = 2016
in17['Year'] = 2017
in18['Year'] = 2018
in19['Year'] = 2019
in20['Year'] = 2020
in21['Year'] = 2021

inYear = pd.concat([in16, in17, in18, in19, in20, in21], axis=0, ignore_index= True)

In [11]:
fips = pd.read_csv("fips.csv")
fips = fips.rename(columns={' st': 'st'})
fips_tup = list(zip(fips.stname, fips.st))

In [12]:
fipsnum = []
for state in inYear['State']:
    for tup in fips_tup:
        if state == tup[0]:
            fipsnum.append(tup[1])

inYear['State FIPS'] = fipsnum

In [13]:
zipdf2 = list(zip(bp_df2['Survey_Date'], bp_df2['State_FIPS']))
zipinYear = list(zip(inYear['Year'], inYear['State FIPS']))

income = []
state = []
for val in zipdf2:
    if val in zipinYear:
        result = zipinYear.index(val)
        income.append(inYear['Mean Monthly Income'].iloc[result])
        state.append(inYear['State'].iloc[result])
bp_df2['Income'] = income
bp_df2['State'] = state
bp_df2['Affordability'] = (bp_df2['Monthly_Mortgage'] / bp_df2['Income']) * 100

bp_df2['State_FIPS'] = bp_df2['State_FIPS'].astype(str).str.zfill(2)
bp_df2['County_FIPS'] = bp_df2['County_FIPS'].astype(str).str.zfill(3)
bp_df2['County'] = bp_df2['State_FIPS'] + bp_df2['County_FIPS']

In [14]:
#final data set
bp_df2

,Survey_Date,State_FIPS,County_FIPS,County_Name,1_unit_Units,1_unit_Value,Mean_unit_Value,Monthly_Mortgage,Income,State,Affordability,County
0,2016,01,001,Autauga County,169,45256130,2.677878e+05,992,3251,Alabama,30.513688,01001
1,2016,01,007,Bibb County,12,2541940,2.118283e+05,784,3251,Alabama,24.115657,01007
2,2016,01,009,Blount County,14,2815172,2.010837e+05,745,3251,Alabama,22.916026,01009
3,2016,01,021,Chilton County,24,3398050,1.415854e+05,524,3251,Alabama,16.118118,01021
4,2016,01,073,Jefferson County,1500,375728598,2.504857e+05,928,3251,Alabama,28.545063,01073
...,...,...,...,...,...,...,...,...,...,...,...,...
4462,2021,55,109,St. Croix County ...,541,169761499,3.137920e+05,1162,4880,Wisconsin,23.811475,55109
4463,2021,55,131,Washington County ...,395,151143081,3.826407e+05,1417,4880,Wisconsin,29.036885,55131
4464,2021,56,025,Natrona County ...,140,36761209,2.625801e+05,972,5468,Wyoming,17.776152,56025
4465,2021,56,029,Park County ...,246,74978563,3.047909e+05,1129,5468,Wyoming,20.647403,56029


In [15]:
#sending dataframe to csv
bp_df2.to_csv('Affordability.csv')